When aplly binary classifiers for multi-class, we will face some problems. The one-vs-rest technique has a downside of the sum of probability. Softmax Regresion can effectively solve these shortcoming

The data x has dimension d+1 (add 1 for bias). Consider sigmoid function:

$ a_i = sigmoid(z_i) = sigmoid(w_i^{T}\textbf{x}) $

In this technique, $ a_i$, i = 1, 2, ..., C is conducted using $ z_i$ only. However this approach does not create any relation between $ a_i$ and $z_i$

Note that $ x_i*w_j = z_j$ of $x_i$

So, let's consider Softmax function:

$ a_i = \frac{exp(z_i)}{\sum_{j=1}^{C}exp(z_j)}$

This function satisfies:

(i) Compute $a_i $ based on every $ z_i$

(ii) $ a_i$ is positive and their sum is equal to 1

(iii) $ a_i$ is propotional with $ z_i$

Let's create softmax function in Python:

In [11]:
import numpy as np

def softmax(Z):
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True))
    return e_Z/e_Z.sum(axis = 0)

In the code above, we subtract every z_i in Z by the maximum z*. This help prevent overflow and reduct the calculation.

To build the loss function, we use Cross Entrophy:

$H(p, q) = -\sum_{i=1}^{C}p_i\log(q_i)$

Note that $H(p, q)$ differs from $H(q, p)$ and $q_i$ > 0. In this case, we use $p$ to represent the real output, which has only 1 value and 0 value for other, and $q$ to represent predition, which always strictly greater than 0 and smaller than 1.

Consider the loss function for the softmax regression:

$J(\textbf{W};\textbf{X};\textbf{Y}) = -\sum_{i=1}^{N}\sum_{j=1}^{C}y_{ij}\log(a_{ij})$

Using Stochastic Gradient Descent with a single point ($x_i, y_i$):

$-\sum_{j=1}^{C}y_{ji}W_j^Tx_i + \log(\sum_{k=1}^{C}\exp(w_k^Tx_i))$

After that, we differentiate the loss function:

$\frac{\partial J_i(W)}{\partial W} = [\frac{\partial J_i(W)}{\partial w_1}, \frac{\partial J_i(W)}{\partial w_2}, ..., \frac{\partial J_i(W)}{\partial w_C}]$

Note that gradient of each columns is:

$\frac{\partial J_i(W)}{\partial w_j} = e_{ji}x_i$ where $ e_{ji} = a_{ji} - y_{ji} $

In conclusion, we have $\frac{\partial J_i(W)}{\partial W} = x_i[e_{1i}, e_{2i}, ..., e_{Ci}] = x_ie_i^T$

$ \frac{\partial J(W)}{\partial W} = \sum_{i=1}^{N}x_ie_i^T = XE^T$ where $E = A-Y$

Update $ W = W+\eta x_i(y_i-a_i)^T$

In [12]:
import numpy as np

N = 2
d = 2
C = 3

X = np.random.randn(d, N)
y = np.random.randint(0, 3, (N,))

In [22]:
## One-hot coding  # review this code
from scipy import sparse 
def convert_labels(y, C = C):
    Y = sparse.coo_matrix((np.ones_like(y), 
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y 

Y = convert_labels(y, C)
print(Y)

[[1 0]
 [0 1]
 [0 0]]


In [26]:
W_init = np.random.randn(d, C)
print(W_init)
#print(np.max(W_init, axis = 0, keepdims = True))

[[ 0.20186097 -1.57735818  2.13312996]
 [ 0.52527968 -0.57292201 -1.78832804]]


In [29]:
print(softmax(W_init.T.dot(X)))

[[0.02783939 0.14235749]
 [0.00196556 0.03276953]
 [0.97019505 0.82487299]]


Now we need to revalidate the derivative:

In [36]:
def cost(X, Y, W):
    A = softmax(W.T.dot(X))
    return -np.sum(Y*np.log(A))
print(cost(X, Y, W_init))

6.999559724142033


In [37]:
def grad(X, Y, W):
    A = softmax(W.T.dot(X))
    E = A - Y
    return X.dot(E.T)

def numerical_grad(X, Y, W, cost):
    eps = 1e-6
    g = np.zeros_like(W)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W_p = W.copy()
            W_n = W.copy()
            W_p[i, j] += eps 
            W_n[i, j] -= eps
            g[i,j] = (cost(X, Y, W_p) - cost(X, Y, W_n))/(2*eps)
    return g

g1 = grad(X, Y, W_init)
g2 = numerical_grad(X, Y, W_init, cost)
print(np.linalg.norm(g1-g2))

3.035166276268107e-10


In [39]:
# Main function for Softmax Regression
def SoftmaxRegression(X, y, W_init, eta, max_iter = 10000):
    W = W_init
    C = W_init.shape[1]
    d = X.shape[0]
    N = X.shape[1]
    Y = convert_labels(y, C)
    iter = 0 # Count the number of iters
    check_w_after = 20
    while iter < max_iter:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(W.T.dot(xi))
            W_new = W + eta*xi.dot((yi-ai).T)
            iter += 1
            if iter%check_w_after == 0:
                if np.linalg.norm(W_new - W)<1e-4:
                    return W_new
                W = W_new
    return W
eta = 0.05

W = SoftmaxRegression(X, y, W_init, eta)
print(W)

[[ 1.35457126  0.96018019 -1.5571187 ]
 [-0.00383276 -0.37226664 -1.45987098]]


In [40]:
# Predict function
def pred(W, X):
    A = softmax_stable(W.T.dot(X))
    return np.argmax(A, axis = 0)

Note: boundary created by softmax regression is linear. This is because a point x is predicted to be a member of class k if $a_j \leq a_k, \forall j \neq k$